In [1]:
#import stuff we need
import numpy as np
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt
import cv2
import os
import copy 
import torch
import torch.nn as nn
import pickle
import math
from routines import *


import warnings
warnings.filterwarnings('ignore')

#np.random.seed(0)
torch.manual_seed(0)

In [2]:
class EnvironmentInvertedPendulum:
    #initialization of internal state
    def __init__(self,m=1,l=1,g=1,max_steps=200,dt=0.005,add_noise=0.01,bottom_up=False):
        self.m=m
        self.l=l
        self.g=g
        self.dt=dt
        self.steps_left=max_steps
        self.reset(add_noise=add_noise)
        self.bottom_up=bottom_up
        if bottom_up:
            self.reset(theta=0+add_noise*np.random.rand())
            self.passed=False
        else:
            self.reset(add_noise=add_noise)

    def reset(self,theta=np.pi,theta_d=0,add_noise=0):
        self.state=[theta+add_noise*np.random.rand(),theta_d+add_noise*np.random.rand()]
        self.state_history=[self.state.copy()]
        
    #returns the current environment's observation to the agent
    def get_observation(self):
        return self.state
    
    #allows the agent to query the set of actions it can execute
    def sample_action(self):
        #return self.state[3]+0.1*(np.random.rand()-0.5)
        return 1.2
    
    def get_energy(self,state):
        e_kin=self.m/2*(self.l*state[1])**2
        e_pot=-self.m*self.g*self.l*np.cos(state[0])
        return e_kin,e_pot
    
    #signals the end of the episode to the agent
    def is_done(self):
        return self.steps_left==0
    
    #central piece: handles agents action and returns reward for the action
    def action(self,action):
        if self.is_done():
            raise Exception('Game is over')
        self.step(action)
        delta=np.abs(self.state[0]-np.pi)
        if self.bottom_up:
            e_pot,e_kin=self.get_energy(self.state)
            e_tot=e_pot+e_kin
            if e_tot>10:
                #energy constraint:
                self.steps_left=0
                return 0
            if delta<1:
                self.passed=True
            if self.passed:
                if delta<np.pi/2:
                    self.steps_left -=1
                    return 10-delta**2
                else:
                    self.steps_left=0
                    return 0
            else:
                self.steps_left -=1
                return 3-delta**2
        else:
            if delta<np.pi/4:
                self.steps_left -=1
                return 2-delta**2
            else:
                self.steps_left=0
                return 0
    
    def explicite_euler(self,dt,state,F=0):
        theta_dd=self.get_theta_dd(state,F)
        next_state= [state[0]+dt*state[1],state[1]+dt*theta_dd]
        return next_state
    
    #the decoupled equations of motion
    def get_theta_dd(self,state,F=0):
        return F/(self.m*self.l)-self.g*np.sin(state[0])/self.l

    
    #differentail time step using explicite midpoint method
    def step(self,F):
        next_state=self.explicite_euler(self.dt/2,self.state,F)
        theta_dd=self.get_theta_dd(next_state,F)
        self.state[0]+=self.dt*next_state[1]
        self.state[1]+=self.dt*theta_dd
        self.state_history.append(self.state.copy())
        
    def render(self,img_res=1,save_path='trash_figures/inverted_pendulum.avi'):
        frames_per_second=20
        take_frame_every=int(1/(self.dt*frames_per_second))
        frames=[]
        h=int(img_res*200)
        w=h
        x0=int(w/2)
        y0=int(h/2)
        h_red=int(0.4*h)
        L=h_red
        d=int(0.02*h)
        d=d*self.m**(1/3)
        #max_theta2_d=1.2*np.max(np.abs(phase_traject[:,3]))
        for i,state_i in enumerate(self.state_history):
            if i%5000==0:
                print('rendering iteration: '+str(i)+'/'+str(len(self.state_history)))           
            if i%take_frame_every==0:
                theta=state_i[0]
                #----transform to cartesian coordinates---
                x1=x0+L*np.sin(theta)
                y1=y0+L*np.cos(theta)
                #---draw the image ----
                img = Image.new("RGB", (w, h), "white")
                draw = ImageDraw.Draw(img)
                draw.line([(x0,y0),(x1,y1)],fill=(0,0,0),width=1)
                draw.ellipse([(x1-d,y1-d),(x1+d,y1+d)], fill=(0,0,0), outline=None)
                frames.append(img)
        cv2_list=self.pil_list_to_cv2(frames)
        self.generate_video(cv2_list,path=save_path,fps=1000/40)
    
    #calculates the potential and kinetic energy of the two masses at a given state
    def get_energy(self,state):
        theta=state[0]
        theta_d=state[1]
        y=self.l*np.cos(theta)
        e_pot=-self.m*self.g*y
        e_kin=self.m/2*(self.l*theta_d)**2
        return e_pot,e_kin
    
    #used for video converting
    def pil_list_to_cv2(self,pil_list):
        #converts a list of pil images to a list of cv2 images
        png_list=[]
        for pil_img in pil_list:
            pil_img.save('trash_image.png',format='png')
            png_list.append(cv2.imread('trash_image.png'))
        os.remove('trash_image.png')
        return png_list

    def generate_video(self,cv2_list,path='car_race.avi',fps=10): 
        #makes a video from a given cv2 image list
        if len(cv2_list)==0:
            raise ValueError('the given png list is empty!')
        video_name = path
        frame=cv2_list[0] 
        # setting the frame width, height width 
        # the width, height of first image 
        height, width, layers = frame.shape   
        video = cv2.VideoWriter(video_name, 0, fps, (width, height))  
        # Appending the images to the video one by one 
        for cv2_image in cv2_list:  
            video.write(cv2_image) 
        # Deallocating memories taken for window creation 
        cv2.destroyAllWindows()  
        video.release()  # releasing the video generated 
            

In [3]:
class EnvironmentInvertedDoublePendulum:
    #initialization of internal state
    def __init__(self,m1=1,m2=1,l1=1,l2=1,g=1,max_steps=200,dt=0.005,add_noise=0.01,bottom_up=False):
        self.m1=m1
        self.m2=m2
        self.l1=l1
        self.l2=l2
        self.g=g
        self.dt=dt
        self.steps_left=max_steps
        self.bottom_up=bottom_up#true if you want to play the bottom up mode
        if bottom_up:
            self.passed=False#store if the outer pendulum has managed to surpass abs(np.pi)
            self.reset(theta1=0+add_noise*np.random.rand(),theta2=0+add_noise*np.random.rand())
        else:
            self.reset(add_noise=add_noise)
            

        
    def reset(self,theta1=np.pi,theta2=np.pi,theta1_d=0,theta2_d=0,add_noise=0):
        self.state=[theta1+add_noise*np.random.rand(),theta2+add_noise*np.random.rand(),theta1_d+add_noise*np.random.rand(),theta2_d+add_noise*np.random.rand()]
        self.state_history=[self.state.copy()]
        
    #returns the current environment's observation to the agent
    def get_observation(self):
        return self.state
    
    #allows the agent to query the set of actions it can execute
    def sample_action(self):
        #return self.state[3]+0.1*(np.random.rand()-0.5)
        return 1.2

    
    #signals the end of the episode to the agent
    def is_done(self):
        return self.steps_left==0
    
    #central piece: handles agents action and returns reward for the action
    def action(self,action):
        if self.is_done():
            raise Exception('Game is over')
        self.step(action)
        delta1=np.abs(self.state[1]%(2*np.pi)-np.pi)
        if self.bottom_up:
            e_pot,e_kin=self.get_energy(self.state)
            e_tot=np.sum(e_pot)+np.sum(e_kin)
            if e_tot>1e3:
                #energy constraint:
                self.steps_left=0
                return 0
            if delta1<0.2:
                self.passed=True
            if self.passed:
                delta0=np.abs(self.state[0]-np.pi)
                if delta1<np.pi/4 and delta0<np.pi/2:
                    self.steps_left -=1
                    return 10-delta1**2-0.01*delta0
                else:
                    self.steps_left=0
                    #self.steps_left -=1#if you want to play the game until max step reached
                    return 0
            else:
                self.steps_left -=1
                return 3-delta1**2
        else:
            delta0=np.abs(self.state[0]%(2*np.pi)-np.pi)
            if delta1<np.pi/4 and delta0<np.pi/2:
                self.steps_left -=1
                return 2-delta1**2
            else:
                self.steps_left=0
                #self.steps_left -=1#if you want to play the game until max step reached
                return 0
    
    def explicite_euler(self,dt,state,F=0):
        theta1_dd,theta2_dd=self.get_theta_dd(state,F)
        next_state= [state[0]+dt*state[2],state[1]+dt*state[3],state[2]+dt*theta1_dd,state[3]+dt*theta2_dd]
        return next_state
    
    #the decoupled equations of motion
    def get_theta_dd(self,state,F=0):
        theta1=state[0]
        theta2=state[1]
        theta1_d=state[2]
        theta2_d=state[3]
        #----theta1_dd-----
        num1=-self.g*((2*self.m1+self.m2)*np.sin(theta1)+self.m2*np.sin(theta1-2*theta2))
        num2=-2*np.sin(theta1-theta2)*self.m2*(theta2_d**2*self.l2+theta1_d**2*self.l1*np.cos(theta1-theta2))
        num3=2*F
        denum1=2*self.m1+self.m2-self.m2*np.cos(2*theta1-2*theta2)
        denum=self.l1*denum1
        theta1_dd=(num1+num2+num3)/denum
        #----theta2_dd----
        num1=2*np.sin(theta1-theta2)
        num2=theta1_d**2*self.l1*(self.m1+self.m2)+self.g*(self.m1+self.m2)*np.cos(theta1)+theta2_d**2*self.l2*self.m2*np.cos(theta1-theta2)
        num3=-2*F*np.cos(theta1-theta2)
        denum=self.l2*denum1
        theta2_dd=(num3+num1*num2)/denum
        return theta1_dd,theta2_dd
    
    #differentail time step using explicite midpoint method
    def step(self,F):
        next_state=self.explicite_euler(self.dt/2,self.state,F)
        theta1_dd,theta2_dd=self.get_theta_dd(next_state,F)
        self.state[0]+=self.dt*next_state[2]
        self.state[1]+=self.dt*next_state[3]
        self.state[2]+=self.dt*theta1_dd
        self.state[3]+=self.dt*theta2_dd
        self.state_history.append(self.state.copy())
        
    
    def render(self,img_res=1,save_path='trash_figures/inverted_double_pendulum.avi'):
        frames_per_second=20
        take_frame_every=int(1/(self.dt*frames_per_second))
        frames=[]
        h=int(img_res*200)
        w=h
        x0=int(w/2)
        y0=int(h/2)
        h_red=int(0.4*h)
        l_tot=self.l1+self.l2
        l1_ratio=self.l1/l_tot
        l2_ratio=self.l2/l_tot
        L1=l1_ratio*h_red
        L2=l2_ratio*h_red
        d=int(0.02*h)
        d1=d*self.m1**(1/3)
        d2=d*self.m2**(1/3)
        d_4=d/4
        #max_theta2_d=1.2*np.max(np.abs(phase_traject[:,3]))
        for i,state_i in enumerate(self.state_history):
            if i%5000==0:
                print('rendering iteration: '+str(i)+'/'+str(len(self.state_history)))           
            if i%take_frame_every==0:
                theta1=state_i[0]
                theta2=state_i[1]
                #----transform to cartesian coordinates---
                x1=x0+L1*np.sin(theta1)
                y1=y0+L1*np.cos(theta1)
                x2=x1+L2*np.sin(theta2)
                y2=y1+L2*np.cos(theta2)
                #---draw the image ----
                img = Image.new("RGB", (w, h), "white")
                draw = ImageDraw.Draw(img)
                draw.line([(x0,y0),(x1,y1)],fill=(0,0,0),width=1)
                draw.ellipse([(x1-d1,y1-d1),(x1+d1,y1+d1)], fill=(0,0,0), outline=None)
                draw.line([(x1,y1),(x2,y2)],fill=(0,0,0),width=1)
                draw.ellipse([(x2-d2,y2-d2),(x2+d2,y2+d2)], fill=(0,0,255), outline=None)
                frames.append(img)
        cv2_list=self.pil_list_to_cv2(frames)
        self.generate_video(cv2_list,path=save_path,fps=1000/40)
    
    #calculates the potential and kinetic energy of the two masses at a given state
    def get_energy(self,state):
        theta1=state[0]
        theta2=state[1]
        theta1_d=state[2]
        theta2_d=state[3]
        y1=self.l1*np.cos(theta1)
        y2=y1+self.l2*np.cos(theta2)
        e_pot=np.array([-self.m1*self.g*y1,-self.m2*self.g*y2])
        e_kin_1=self.m1/2*(self.l1*theta1_d)**2
        e_kin_2=(self.l1*theta1_d)**2
        e_kin_2+=(self.l2*theta2_d)**2
        e_kin_2+=2*self.l1*self.l2*theta1_d*theta2_d*(np.cos(theta1)*np.cos(theta2)+np.sin(theta1)*np.sin(theta2))
        e_kin_2*=self.m2/2
        e_kin=np.array([e_kin_1,e_kin_2])
        return e_pot,e_kin
    
    #used for video converting
    def pil_list_to_cv2(self,pil_list):
        #converts a list of pil images to a list of cv2 images
        png_list=[]
        for pil_img in pil_list:
            pil_img.save('trash_image.png',format='png')
            png_list.append(cv2.imread('trash_image.png'))
        os.remove('trash_image.png')
        return png_list

    def generate_video(self,cv2_list,path='car_race.avi',fps=10): 
        #makes a video from a given cv2 image list
        if len(cv2_list)==0:
            raise ValueError('the given png list is empty!')
        video_name = path
        frame=cv2_list[0] 
        # setting the frame width, height width 
        # the width, height of first image 
        height, width, layers = frame.shape   
        video = cv2.VideoWriter(video_name, 0, fps, (width, height))  
        # Appending the images to the video one by one 
        for cv2_image in cv2_list:  
            video.write(cv2_image) 
        # Deallocating memories taken for window creation 
        cv2.destroyAllWindows()  
        video.release()  # releasing the video generated 
            

In [4]:
class GeneticAgent:
    #initialize the counter for the total reward
    def __init__(self,n_neurons,bottom_up=False):
        self.total_reward=0.0
        self.bottom_up=bottom_up
        self.initialize_policy(n_neurons)
            
    #accepts the environment instance as an argument and allows the agents to observe and act
    def step(self,env):
        observation=env.get_observation()
        #action = env.sample_action()
        action=self.get_action(observation)
        reward=env.action(action)
        self.total_reward+=reward
        
    def reset_reward(self):
        self.total_reward=0
        
    def get_action(self,state):
        #normalization
        s=state.copy()
        #if not self.bottom_up:
            #if len(s)==2:
                #s[0]-=np.pi
            #elif len(s)==4:
                #s[0]-=np.pi
                #s[1]-=np.pi
        with torch.no_grad():
            F=self.policy(torch.FloatTensor(s)).item()
            return 10*F
        
    def mutate(self,muatation_rate=0.1):
        with torch.no_grad():
            for param in self.policy.parameters():
                param.add_(torch.randn(param.size()) * muatation_rate)
        
    def initialize_policy(self,n_neurons):
        self.policy=self.get_nn(n_neurons)
    
    def get_nn(self,n_neurons):
        neural_network=nn.Sequential()
        if len(n_neurons)<2:
            raise ValueError('n_neurons must contain at least two entries for in- and output')
        depth=len(n_neurons)-2
        for i in range(depth):
            neural_network.add_module("layer"+str(i),nn.Sequential(nn.Linear(n_neurons[i],n_neurons[i+1]),nn.Sigmoid()))
        neural_network.add_module("layer"+str(depth),nn.Sequential(nn.Linear(n_neurons[depth],n_neurons[depth+1])))
        return neural_network
        


In [5]:
#functions for genetic algorithm
def get_first_generation(population_size,agent,n_neurons,load_best_agent=False):
    if load_best_agent:
        agent=pickle.load(open('models/best_agent.pkl', "rb" ))
    agent_list=[]
    for i in range(population_size):
        new_agent=copy.deepcopy(agent)
        new_agent.reset_reward()
        if load_best_agent:
            if i>=1:
                new_agent.mutate(muatation_rate=0.1)
        else:
            new_agent.initialize_policy(n_neurons)
        agent_list.append(new_agent)
    return agent_list

def get_scores(agent_list,environment):
    scores=[]
    environments=[]
    for agent in agent_list:
        env=copy.deepcopy(environment)
        while not env.is_done():
            agent.step(env)
        scores.append(agent.total_reward)
        environments.append(env)
    return scores,environments

def get_next_generation(scores,agent_list,n_survivors,population_size,muatation_rate=0.1,store_best_agent=True):
    np_scores=np.asarray(scores)
    rank_idx=np.argsort(np_scores)
    agent_elite=[]
    for i in range(n_survivors):
        agent_elite.append(agent_list[rank_idx[-i-1]])
    if store_best_agent:
        pickle.dump(agent_elite[0],open('models/best_agent.pkl', "wb" ))
    new_agent_list=[]
    for j in range(population_size):
        new_agent=copy.deepcopy(agent_elite[j%n_survivors])
        if j>=1:
            new_agent.mutate(muatation_rate)
        new_agent.reset_reward()
        new_agent_list.append(new_agent)
    return new_agent_list,rank_idx
    

In [12]:
def get_largest_length(list_of_lists):
    l=0
    for list in list_of_lists:
        if len(list)>l:
            l=len(list)
    return l

#functions for video presentaiton
def render_single_pendulum(state_history_list,dt,best_idx=None,m=1,img_res=1.5,save_path='trash_figures/inverted_pendulum_generation.avi'):
    frames_per_second=20
    take_frame_every=int(1/(dt*frames_per_second))
    frames=[]
    h=int(img_res*200)
    w=h
    x0=int(w/2)
    y0=int(h/2)
    h_red=int(0.4*h)
    L=h_red
    d=int(0.02*h)
    d=d*m**(1/3)
    if best_idx is not None:
        state_history_list.append(state_history_list[best_idx])
    largest_trajectory=get_largest_length(state_history_list)
    for i in range(largest_trajectory):
        img = Image.new("RGB", (w, h), "white")
        draw = ImageDraw.Draw(img)
        for j in range(len(state_history_list)):     
            if i%take_frame_every==0 and len(state_history_list[j])>i:
                if best_idx is not None:
                    if j==len(state_history_list)-1:
                        color=(255,0,0)
                    else:
                        color=(0,0,0)
                theta=state_history_list[j][i][0]
                #----transform to cartesian coordinates---
                x1=x0+L*np.sin(theta)
                y1=y0+L*np.cos(theta)
                #---draw the image ----
                draw.line([(x0,y0),(x1,y1)],fill=(0,0,0),width=1)
                draw.ellipse([(x1-d,y1-d),(x1+d,y1+d)], fill=color, outline=None)
        frames.append(img)
    cv2_list=pil_list_to_cv2(frames)
    generate_video(cv2_list,path=save_path,fps=1000/40)
    
def render_double_pendulum(state_history_list,dt,best_idx=None,m1=1,m2=1,l1=1,l2=1,img_res=1.5,save_path='trash_figures/inverted_double_pendulum_generation.avi'):
    frames_per_second=20
    take_frame_every=int(1/(dt*frames_per_second))
    frames=[]
    h=int(img_res*200)
    w=h
    x0=int(w/2)
    y0=int(h/2)
    h_red=int(0.4*h)
    l_tot=l1+l2
    l1_ratio=l1/l_tot
    l2_ratio=l2/l_tot
    L1=l1_ratio*h_red
    L2=l2_ratio*h_red
    d=int(0.02*h)
    d1=d*m1**(1/3)
    d2=d*m2**(1/3)
    d_4=d/4
    if best_idx is not None:
        state_history_list.append(state_history_list[best_idx])
    largest_trajectory=get_largest_length(state_history_list)
    for i in range(largest_trajectory):
        img = Image.new("RGB", (w, h), "white")
        draw = ImageDraw.Draw(img)
        for j in range(len(state_history_list)):     
            if i%take_frame_every==0 and len(state_history_list[j])>i:
                if best_idx is not None:
                    if j==len(state_history_list)-1:
                        color1=(255,0,0)
                        color2=(255,0,0)
                    else:
                        color1=(0,0,255)
                        color2=(0,0,0)
                theta=state_history_list[j][i][0]
                theta1=state_history_list[j][i][0]
                theta2=state_history_list[j][i][1]
                #----transform to cartesian coordinates---
                x1=x0+L1*np.sin(theta1)
                y1=y0+L1*np.cos(theta1)
                x2=x1+L2*np.sin(theta2)
                y2=y1+L2*np.cos(theta2)
                #---draw the image ----
                draw.line([(x0,y0),(x1,y1)],fill=color2,width=1)
                draw.ellipse([(x1-d1,y1-d1),(x1+d1,y1+d1)], fill=color1, outline=None)
                draw.line([(x1,y1),(x2,y2)],fill=color2,width=1)
                draw.ellipse([(x2-d2,y2-d2),(x2+d2,y2+d2)], fill=color1, outline=None)
        frames.append(img)
    cv2_list=pil_list_to_cv2(frames)
    generate_video(cv2_list,path=save_path,fps=1000/40)
    
def pil_list_to_cv2(pil_list):
    #converts a list of pil images to a list of cv2 images
    png_list=[]
    for pil_img in pil_list:
        pil_img.save('trash_image.png',format='png')
        png_list.append(cv2.imread('trash_image.png'))
    os.remove('trash_image.png')
    return png_list

def generate_video(cv2_list,path='car_race.avi',fps=10): 
    #makes a video from a given cv2 image list
    if len(cv2_list)==0:
        raise ValueError('the given png list is empty!')
    video_name = path
    frame=cv2_list[0] 
    # setting the frame width, height width 
    # the width, height of first image 
    height, width, layers = frame.shape   
    video = cv2.VideoWriter(video_name, 0, fps, (width, height))  
    # Appending the images to the video one by one 
    for cv2_image in cv2_list:  
        video.write(cv2_image) 
    # Deallocating memories taken for window creation 
    cv2.destroyAllWindows()  
    video.release()  # releasing the video generated 

In [15]:
n_generations=30
population_size=20
n_survivors=3
n_neurons=[4,10,1]
muatation_rate=0.1
add_noise=0.01
max_steps=500
bottom_up=True
make_video=False

agent=GeneticAgent(n_neurons,bottom_up=bottom_up)
new_agents=get_first_generation(population_size,agent,n_neurons,load_best_agent=True)#all agents initialized independently at random
running_score=0
for i in range(n_generations):
    state_history_list=[]
    #bottom_up=0.5>np.random.rand()
    #print('bottom up: '+str(bottom_up))
    if n_neurons[0]==2:
        env=EnvironmentInvertedPendulum(max_steps=max_steps,dt=0.03,m=1,add_noise=add_noise,bottom_up=bottom_up)
    else:
        env=EnvironmentInvertedDoublePendulum(max_steps=max_steps,dt=0.03,m1=0.5,m2=1,l1=1,l2=1,add_noise=add_noise,bottom_up=bottom_up)
        if bottom_up:
            env.reset(theta1=0+add_noise*np.random.rand(),theta2=0+add_noise*np.random.rand())
    scores,environments=get_scores(new_agents,env)
    new_agents,rank_idx=get_next_generation(scores,new_agents,n_survivors,population_size,muatation_rate,store_best_agent=True)
    best_score=scores[rank_idx[-1]]
    for environment in environments:
        state_history_list.append(environment.state_history)
    if make_video:
        if n_neurons[0]==2:
            render_single_pendulum(state_history_list,best_idx=rank_idx[-1],dt=0.03,m=1,img_res=1.5,save_path='inverted_pendulum/inverted_pendulum_gen='+str(i)+'.avi')
        else:
            render_double_pendulum(state_history_list,best_idx=rank_idx[-1],dt=0.03,m1=0.5,m2=1,l1=1,l2=1,img_res=1.5,save_path='inverted_double_pendulum/inverted_double_pendulum_gen='+str(i)+'.avi')

    print('generation: '+str(i+1)+', best score: '+str(best_score))
#---render epoch--
#print('render environment...')
#environments[rank_idx[-1]].render(save_path='inverted_pendulum/inverted_pendulum')




#---check the energy
#e_pot_start,e_kin_start=env.get_energy(env.state_history[0])
#e_pot_end,e_kin_end=env.get_energy(env.state_history[-1])
#e_start=np.sum(e_pot_start)+np.sum(e_kin_start)
#e_end=np.sum(e_pot_end)+np.sum(e_kin_end)
#print('starting energy: '+str(e_start))
#print('final energy: '+str(e_end))





generation: 1, best score: 157.42216298416497
generation: 2, best score: 184.6745874705414
generation: 3, best score: 207.5024253231064
generation: 4, best score: 465.3568384917641
generation: 5, best score: 465.3500179284539
generation: 6, best score: 455.3366472036032
generation: 7, best score: 445.48346510969355
generation: 8, best score: 455.49379962754256
generation: 9, best score: 465.2216215896461
generation: 10, best score: 455.15886173209236
generation: 11, best score: 465.52523124085536
generation: 12, best score: 455.36485301793243
generation: 13, best score: 465.1343227683871
generation: 14, best score: 455.52201242040917
generation: 15, best score: 465.28283871742775
generation: 16, best score: 497.8898514285825
generation: 17, best score: 271.74559230320983
generation: 18, best score: 271.3445436182375
generation: 19, best score: 271.17266770594193
generation: 20, best score: 271.2135604240242
generation: 21, best score: 295.91721453043357
generation: 22, best score: 287.